# Regresión Logística

- Modelo Supervizado de CLASIFICACIÓN.

Un criterio para modelizar el dataset es plantear que de acuerdo a la posición de un patrón en particular en el plano, ese patrón tiene una probabildad P de pertenecer a la clase 1 y 1-P de pertenecer a la clase 2.

**La idea es que la transición de la zona en la que los patrones pertenecen a la clase 1 y la zona en la que los patrones pertenecen a la clase 2, tenga una transición suave.**


Una hipótesis para nuestro problema podría ser que un patrón en particular (x1,x2) tiene una probabilidad de ser :

$$ P(h) = \frac{1}{1 + e^{-h}} $$

A la función definida anteriormente se la llama sigmoide y tiene la siguiente forma.

<img src="../Imagenes/funcion_sigmoide.png" width=50%>

Cuando h=0, habrá una zona en la cual la probabilidad de que un patrón sea de la clase 1 es 0.5. Este valor definirá el umbral entre los patrones de tipo 1 y los patrones de tipo 2.  

Si queremos que para nuestro problema de clasificación dicho umbral sea una recta, podemos definir:

$$ h= \beta_0+\beta_1*x_1+\beta_2*x_2 $$

Nótese que cuando h=0, la probabilidad de un patrón de pertenecer a la clase 1, es 0.5. Esto ocurre en el espacio geométrico:

$$ x_2=-\frac{\beta_1}{\beta_2}.x_1-\frac{\beta_0}{\beta_2} $$

Estamos proponiendo un modelo probabilístico de parámetros $\beta_1,\beta_2,\beta_0$ y por lo tanto la pregunta es:

** ¿Cuáles son los valores de $\beta_1,\beta_2,\beta_0$ que maximizan el likelihood de este modelo?**

Para ello, podemos plantear el likelihood del modelo como la probabilidad de que este modelo haya generado los patrones observados. Si suponemos que las observaciones son independientes nos queda:

$$\mathcal{L} = \prod_{i=1}^{N} Pr(Y=y_i|X=x_i) = \prod_{i=1}^{N} p(x_i;\beta_1,\beta_2,\beta_0)^{y_i}.[1 -p(x_i;\beta_1,\beta_2,\beta_0)]^{1 - y_i} = $$
$$=\prod_{i=1}^{N} \left(\frac{1}{1 + e^{-(\beta_0+\beta_1*x_1+\beta_2*x_2)}}\right)^{y_i} . \left(1- \frac{1}{1 + e^{-(\beta_0+\beta_1*x_1+\beta_2*x_2)}}\right)^{1-y_i} $$

Queremos encontrar los parámetros $\beta_i$ que maximicen el likelihood. Como hemos visto anteriormente, maximizar el likelihood equivale a maximizar el logaritmo de éste, ya que el logaritmo es una función monótona creciente. Una forma de hacerlo es utilizar el metodo de Gradient Descent y minimizar el $-log(\mathcal{L})$. Al $-log(\mathcal{L})$ se lo denomina **cross-entropy** cost function y en general está definido como:

$$-\log(\mathcal{L})=-\sum_{i=1}^{N}\left[y_i.\log{p(x_i)}+(1-y_i)\log{(1-p(x_i))}\right]$$

Para ello debemos calcular el valor de la derivada del $-log(\mathcal{L})$. Se puede demostrar que:

$$\frac{\partial{log(\mathcal{L})}}{\partial{\beta_j}}=\sum_{i=1}^{N}(y_i-P(x_i;\beta_1,\beta_2,\beta_0)).x_{ij}=\sum_{i=1}^{N}\left(y_i-\frac{1}{1 + e^{-(\beta_0+\beta_1*x_1+\beta_2*x_2)}}\right).x_{ij}$$

Tal como en el caso del error cuadrático medio, se puede maximizar el likelihood evaluándolo para todos los datos (Batch GD), para algunos datos (MiniBatch GS) o para un solo dato (Stochastic GD).

In [6]:
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from random import randint
np.set_printoptions(threshold=np.inf) #Print complete matrix/array

In [7]:
def LogisticRegression(feature_matrix, labels, feature_matrix_test):
    lr = linear_model.LogisticRegression(tol=0.00000004)
    
    #Entrenamos el modelo
    lr.fit(feature_matrix, labels)
    
    #Puntaje del modelo
    score = lr.score(feature_matrix, labels)
    
    #Predice el resultado (La categoria resultante)
    predict = lr.predict(feature_matrix_test)
    
    #Devuelve las probabilidades de cada una de las etiquetas
    predict_proba = lr.predict_proba(feature_matrix_test)
    
    return score, predict, predict_proba

## Regresión logística multiclase

Hasta ahora hemos clasificado los patrones de entrada en dos clases.
¿Cómo se puede adaptar lo visto anteriormente en el caso de que $y_i$ pueda pertenecer a K clases?
Para ello se define un estimador de probabilidad para cada una de las categorías.
La probabilidad de que el patrón de entrada $x^i$ pertenezca a una categoría k es:

$$P(Y=k|x_i,\beta)=\frac{e^{\beta^k.x_i}}{\sum_{j=1}^{K}{e^{\beta^j.x_i}}}$$

A esta función se la denomina función **softmax**. Esta ecuación proporciona una transición suave en cuanto a la probabilidad de pertenecer a cada clase.
Nótese que para el caso de K=2, queda la función sigmoidea de parámetro $\beta = \beta^2-\beta^1$

El procedimiento para encontrar los parámetros $\beta$ es el mismo que para la función logística. Se calcula el likelihood del modelo para un grupo de observaciones y se buscan los parámetros que maximizan dicho likelihood, es decir, minimizan la cross-entropy. 

In [17]:
def LogisticMultiRegression(feature_matrix, labels, feature_matrix_test):
    lr = linear_model.LogisticRegression(multi_class='multinomial',solver='lbfgs',max_iter=100,verbose=5,C=0.01)
    
    #Entrenamos el modelo
    lr.fit(feature_matrix, labels)
    
    #Puntaje del modelo
    score = lr.score(feature_matrix, labels)
    
    #Predice el resultado (La categoria resultante)
    predict = lr.predict(feature_matrix_test)
    
    #Devuelve las probabilidades de cada una de las etiquetas
    predict_proba = lr.predict_proba(feature_matrix_test)
    
    return score, predict, predict_proba

## Examples

### SPAM o HAM

In [27]:
labels = np.array(['SPAM', 'HAM', 'HAM', 'SPAM', 'HAM', 'HAM', 'NN'])
vocabulary = np.array(['alargue', 'automóvil', 'casa', 'novedoso'])
# Simulo que lee los mails y obtiene esta cantidad de palabras
train_mails = ["alargue alargue alargue automóvil automóvil automóvil casa novedoso novedoso novedoso novedoso novedoso novedoso", 
        "alargue automóvil automóvil automóvil automóvil casa casa casa casa casa casa casa novedoso",
        "alargue automóvil automóvil automóvil automóvil casa casa casa novedoso",
        "alargue alargue alargue alargue automóvil automóvil casa novedoso novedoso novedoso novedoso novedoso",
        "alargue automóvil automóvil automóvil casa casa casa casa novedoso novedoso",
        "alargue alargue automóvil automóvil automóvil automóvil casa casa casa casa casa novedoso",
        "alargue alargue alargue alargue alargue alargue alargue alargue alargue automóvil casa novedoso"]
test_mails = ["alargue alargue alargue alargue alargue alargue alargue alargue alargue automóvil casa aguila"]

vectorizer = CountVectorizer()
train_matrix = vectorizer.fit_transform(train_mails)
test_matrix = vectorizer.transform(test_mails)

score, predict, predict_proba = LogisticRegression(train_matrix, labels, test_matrix)
percent = round(float(np.amax(predict_proba, axis=1)) * 100, 2)
print("Logistic Regression")
print("Score", score)
print(f"Se predice que el mail es {predict} con un {percent}% de certeza")
print("\n")

score, predict, predict_proba = LogisticMultiRegression(train_matrix, labels, test_matrix)
percent = round(float(np.amax(predict_proba, axis=1)) * 100, 2)
print("Multiclass Logistic Regression")
print("Score", score)
print(f"Se predice que el mail es {predict} con un {percent}% de certeza")

Logistic Regression
Score 1.0
Se predice que el mail es ['NN'] con un 91.5% de certeza


Multiclass Logistic Regression
Score 0.5714285714285714
Se predice que el mail es ['HAM'] con un 44.5% de certeza


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


### Cancer test

In [25]:
data = np.genfromtxt(fname='../datasets/cancer.txt', delimiter=',')
data = np.delete(data, np.s_[0], axis=1)
data = data[~np.isnan(data).any(axis=1)] #Eliminamos los nan

labels = data[:, -1]
features = data[:, :-1]

test = np.matrix([10,5,6,10,6,10,7,7,10])
score, predict, predict_proba = LogisticRegression(features, labels, test)

cancer_type = "benigno" if int(predict) == 2 else "maligno"
percent = round(float(np.amax(predict_proba, axis=1)) * 100, 2)
print("Score: ", score)
print(f"Se predice que la muestra es del tipo {cancer_type} con un {percent}% de certeza")

Score:  0.9692532942898975
Se predice que la muestra es del tipo maligno con un 99.98% de certeza
